In [13]:
!pip install -r requirements.txt

  Using cached asyncio-3.4.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/40.9 MB ? eta -:--:--
    --------------------------------------- 0.5/40.9 MB 1.7 MB/s eta 0:00:25
    --------------------------------------- 0.8/40.9 MB 1.6 MB/s eta 0:00:26
   - -------------------------------------- 1.3/40.9 MB 1.6 MB/s eta 0:00:26
   - -------------------------------------- 1.6/40.9 MB 1.6 MB/s eta 0:00:25
   -- ------------------------------------- 2.1/40.9 MB 1.6 MB/s eta 0:00:24
   -- ------------------------------------- 2.4/40.9 MB 1.6 MB/s eta 0:00:24
   -- ------------------------------------- 2.6/40.9 MB 1.6 MB/s eta 0:00:24
   -- ------------------------------------- 2.9/40.9 MB 1.6 MB/s eta 0:00:24
   --- ------------------------------------ 3.4/40.9 MB 1.6 MB/s eta 0:00:24
   --- -------------------------

In [14]:
import json
import sys
import os
from openai import OpenAI
sys.path.append("src")

In [3]:
%load_ext autoreload
%autoreload 2

In [26]:
%reload_ext autoreload

In [15]:
from wiki_extract import Extracter
from wiki_gen import WikiGen

In [6]:
def get_article(name):
    print('Article name: ', name)
    page = Extracter(name)
    page.get_references()
    page.get_outline()
    page.get_reference_positions()
    page.newspaper_ref()
    page.get_filtered_outline()
    return page

In [7]:
article_names = [
    'Python',
    'Летние Олимпийские игры 2024',
    'Квантовый компьютер',
    'Присоединение Крыма к Российской Федерации',
    'Сколково (инновационный центр)',
    'Tomb Raider (игра, 2013)',
    'Чёрная дыра',
    'Экономика США',
    'Искусственный интеллект',
    'COVID-19',
    'Броненосный крейсер',
    'Применение искусственного интеллекта',
    'РИА Новости'
]

In [8]:
def save_texts(article_name, texts, number):

    directory = os.path.join("Articles", "Sources", article_name)

    os.makedirs(directory, exist_ok=True)
    '''
    for i, text in enumerate(texts, start=number):
        file_path = os.path.join(directory, f"source_{i}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text)
    '''

In [9]:
articles = {}

for name in article_names:
    page = get_article(name)
    articles[name] = page.ref_texts.keys()
    num = 1
    for _, texts in page.ref_texts.items():
        save_texts(page.name, texts, num)
        num += len(texts)

with open('saved_ref.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)

Article name:  COVID-19


Calculating reference positions: 100%|██████████| 57/57 [00:00<00:00, 57332.21it/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Calculating reference positions: 100%|██████████| 57/57 [00:00<00:00, 56963.39it/s]


Article name:  Броненосный крейсер


Calculating reference positions: 100%|██████████| 31/31 [00:00<00:00, 30994.86it/s]


Article name:  Применение искусственного интеллекта


Calculating reference positions: 100%|██████████| 48/48 [00:00<00:00, 48049.31it/s]


Article name:  РИА Новости


Calculating reference positions: 100%|██████████| 31/31 [00:00<?, ?it/s]


In [30]:
oclient = OpenAI(api_key=KEY, base_url=URL)

wiki_writer = WikiGen(oclient)

for name in article_names:

    json_str = wiki_writer.get_subqueries(name)

    if json_str:
        data = json.loads(json_str)
        directory = os.path.join("Generation", "Subqueries", name)
        os.makedirs(directory, exist_ok=True)
        filename = name.replace(" ", "_") + '.json'
        file_path = os.path.join(directory, filename)
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

In [38]:
nli_prompt = """# Тема
{}

# Текст
{}

Релевантен ли текст для данной темы?
Начни ответ с {} или {}
"""
from openai_utils import LlmCompleter
from openai_utils import AsyncList

positive_choice = 'YES'
negative_choice = 'NO'

client = LlmCompleter(URL, KEY,'qwen2.5-72b')
print(await client.client.models.list())
topic = 'COVID-19'
directory = 'Articles/Sources/COVID-19'

texts = []
file_names = []
for file in os.listdir(directory):
    if file.endswith('.txt'):
        file_path = os.path.join(directory, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            texts.append(f.read())
        file_names.append(file)

directory = 'Articles/Sources/Экономика США'
for file in os.listdir(directory):
    if file.endswith('.txt'):
        file_path = os.path.join(directory, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            texts.append(f.read())
        file_names.append(file)

probs = AsyncList()
for i1, text in enumerate(texts):
    #print(i1)
    # Разбиваем текст на окна по 2000 слов с перекрытием 512 слов
    words = text.split()
    window_size = 2000
    overlap = 512
    windows = []
    i = 0
    while i < len(words):
        snippet = ' '.join(words[i: i + window_size])
        windows.append(snippet)
        if i + window_size >= len(words):
            break
        i += window_size - overlap

    # Для каждого сниппета формируем запрос и получаем вероятность
    snippet_probs = AsyncList()
    for k, snippet in enumerate(windows):
        #print("номер сниппета ", k)
        prompt = nli_prompt.format(
            topic,
            snippet,
            positive_choice,
            negative_choice,
        )
        snippet_probs.append(client.get_probability(prompt, rep_penalty=1.0, max_tokens=10))
    
    # Дожидаемся выполнения всех корутин для сниппетов
    await snippet_probs.complete_couroutines(batch_size=20)
    snippet_results = await snippet_probs.to_list()

    # Вычисляем итоговую вероятность для текста как среднее значение по сниппетам
    combined_prob = 0.0
    count = 0
    for sp in snippet_results:
        if negative_choice in sp:
            prob_val = 1 - sp[negative_choice]
        elif positive_choice in sp:
            prob_val = sp[positive_choice]
        else:
            prob_val = 0.0
        combined_prob += prob_val
        count += 1
    if count > 0:
        combined_prob /= count
    print("Текст номер ", i1, "с вероятностью", combined_prob) 
    probs.append([combined_prob, i1])

# будет выполнять запрос в 20 потоков, не стоит ставить больше 40
await probs.complete_couroutines(batch_size=20)
probs = await probs.to_list()

for prob in probs:
    print(prob[0], prob[1])


AsyncPage[Model](data=[Model(id='RefalMachine/RuadaptQwen2.5-32B-Pro-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned'), Model(id='RefalMachine/RuadaptQwen2.5-32B-QWQ-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned'), Model(id='llama3-70b', created=None, object=None, owned_by=None, max_model_len=14000, status='spawned'), Model(id='qwen2.5-72b', created=None, object=None, owned_by=None, max_model_len=14000, status='spawned'), Model(id='RefalMachine/RuadaptQwen2.5-7B-Garant-v2', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded'), Model(id='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded'), Model(id='qwen32b-coder', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded'), Model(id='tlite', created=None, object=None, owned_by=None, max_model_len=-1, status='offloaded')], object=None)